In [16]:
import numpy as np
import pandas as pd
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

In [17]:
raw_data = pd.read_csv("IPL_Ball_by_Ball_2008_2022.csv")
raw_data = raw_data.sample(frac=0.01, random_state=42)
raw_data

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
177814,501226,1,12,2,MS Dhoni,Yuvraj Singh,MEK Hussey,NaN,1,0,1,0,0,NaN,NaN,NaN,Chennai Super Kings
3287,1304107,1,17,4,SW Billings,T Natarajan,AD Russell,NaN,6,0,6,0,0,NaN,NaN,NaN,Kolkata Knight Riders
188612,419148,2,13,3,RV Uthappa,AD Mathews,LRPL Taylor,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
96212,980961,1,13,6,RA Jadeja,Z Khan,KD Karthik,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Lions
30637,1254064,1,5,1,RR Pant,JD Unadkat,AM Rahane,NaN,4,0,4,0,0,NaN,NaN,NaN,Delhi Capitals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129868,729291,2,3,6,CA Pujara,DS Kulkarni,GJ Maxwell,NaN,0,0,0,0,0,NaN,NaN,NaN,Kings XI Punjab
61912,1136618,2,15,2,H Klaasen,SP Narine,SV Samson,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
100936,980921,1,7,1,V Kohli,A Mishra,AB de Villiers,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
140959,598033,1,10,6,KD Karthik,J Syed Mohammad,RG Sharma,NaN,2,0,2,0,0,NaN,NaN,NaN,Mumbai Indians


In [18]:
# def rectify_data(raw_data):
    
#     processed_data = pd.DataFrame()
#     processed_data['first_innings'] = 2-raw_data['innings']
#     processed_data['second_innings'] = raw_data['innings']-1
#     processed_data['powerplay'] = (raw_data['overs']<=6).astype(int)
#     processed_data['middle'] = ((raw_data['overs']>6) & (raw_data['overs']<=15)).astype(int)
#     processed_data['death'] = (raw_data['overs']>15).astype(int)
#     distinct_batters = raw_data['batter'].unique()
#     batter_names = pd.concat([raw_data['batter'].fillna('') == batter for batter in distinct_batters], axis=1).astype(int)
#     batter_names.columns = distinct_batters
#     processed_data = pd.concat([processed_data,batter_names],axis=1)
#     distinct_bowlers = raw_data['bowler'].unique()
#     bowler_names = pd.concat([raw_data['bowler'].fillna('') == bowler for bowler in distinct_bowlers], axis=1).astype(int)
#     bowler_names.columns = distinct_bowlers
#     processed_data = pd.concat([processed_data,batter_names],axis=1)
#     # processed_data['run_and_wicket'] = raw_data[['total_run','isWicketDelivery']].apply(list,axis=1)
#     processed_data['label'] = raw_data['total_run']+7*raw_data['isWicketDelivery']

#     #outliers
#     processed_data = processed_data[processed_data['label']%7<=6]
    
#     return processed_data

In [19]:
def rectify_data(raw_data):
    raw_data=raw_data[raw_data['total_run']<=6]
    processed_data = pd.DataFrame()
    processed_data['first_innings'] = 2-raw_data['innings']
    processed_data['second_innings'] = raw_data['innings']-1
    processed_data['powerplay'] = (raw_data['overs']<=6).astype(int)
    processed_data['middle'] = ((raw_data['overs']>6) & (raw_data['overs']<=15)).astype(int)
    processed_data['death'] = (raw_data['overs']>15).astype(int)
    distinct_batters = {}
    counter = 0
    for i in raw_data['batter'].unique() :
        distinct_batters[i]=counter
        counter+=1
    processed_data['batter'] = [distinct_batters[x] for x in raw_data['batter']]
    distinct_bowlers = {}
    counter = 0
    for i in raw_data['bowler'].unique() :
        distinct_bowlers[i]=counter
        counter+=1
    processed_data['bowler'] = [distinct_bowlers[x] for x in raw_data['bowler']]
    # processed_data['total_run'] = raw_data['total_run']
    # processed_data['iswicket'] = raw_data['isWicketDelivery']
    processed_data['label'] = raw_data['total_run']+7*raw_data['isWicketDelivery']
    return processed_data

In [20]:
modified_data = rectify_data(raw_data)
X = modified_data.drop(columns = "label")
Y = modified_data["label"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 1092003)
len(modified_data.columns)

8

In [21]:
def Logistic_Regression(X_train,Y_train):
    model = LogisticRegression()
    model.fit(X_train,Y_train)
    return model

In [22]:
def SVM(X_train,Y_train):
    model = SVC()
    model.fit(X_train,Y_train)
    return model

In [23]:
def MLP(X_train,Y_train):
    model = MLPClassifier(activation = 'relu',hidden_layer_sizes=(10,5,10), max_iter=30000)
    model.fit(X_train,Y_train)
    return model

In [24]:
print("Using Logistic Regression :-")
model = Logistic_Regression(X_train,Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using Logistic Regression :-
Accuracy on Trained data -  0.4518805309734513
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.43      0.50      0.46       560
           1       0.46      0.73      0.57       742
           2       0.00      0.00      0.00       109
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00       216
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00        83
           7       0.00      0.00      0.00        80
           8       0.00      0.00      0.00         9

    accuracy                           0.45      1808
   macro avg       0.10      0.14      0.11      1808
weighted avg       0.32      0.45      0.38      1808

Confusion matrix on Trained data - 
 [[279 281   0   0   0   0   0   0   0]
 [204 538   0   0   0   0   0   0   0]
 [ 31  78   0   0   0   0   0   0   0]
 [  2   4   0   0   0  

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

In [25]:
print("Using SVM :-")
model = SVM(X_train,Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using SVM :-
Accuracy on Trained data -  0.4103982300884956
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       560
           1       0.41      1.00      0.58       742
           2       0.00      0.00      0.00       109
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00       216
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00        83
           7       0.00      0.00      0.00        80
           8       0.00      0.00      0.00         9

    accuracy                           0.41      1808
   macro avg       0.05      0.11      0.06      1808
weighted avg       0.17      0.41      0.24      1808

Confusion matrix on Trained data - 
 [[  0 560   0   0   0   0   0   0   0]
 [  0 742   0   0   0   0   0   0   0]
 [  0 109   0   0   0   0   0   0   0]
 [  0   6   0   0   0   0   0   0   0]


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metr

In [26]:
print("Using MLP :-")
model = MLP(X_train,Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using MLP :-
Accuracy on Trained data -  0.435287610619469
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.41      0.30      0.35       560
           1       0.44      0.83      0.58       742
           2       0.00      0.00      0.00       109
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00       216
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00        83
           7       0.00      0.00      0.00        80
           8       0.00      0.00      0.00         9

    accuracy                           0.44      1808
   macro avg       0.09      0.13      0.10      1808
weighted avg       0.31      0.44      0.35      1808

Confusion matrix on Trained data - 
 [[170 390   0   0   0   0   0   0   0]
 [125 617   0   0   0   0   0   0   0]
 [ 23  86   0   0   0   0   0   0   0]
 [  1   5   0   0   0   0   0   0   0]
 

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metr